In [ ]:
import datasets
import pandas as pd

dataset = datasets.load_dataset("trl-lib/tldr")


# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset['test'])
print(f"Dataset shape: {df.shape}")
print(df.head())

Generating test split: 100%|██████████| 6553/6553 [00:00<00:00, 37591.99 examples/s]


Dataset shape: (6553, 2)
                                              prompt  \
0  SUBREDDIT: r/relationships\n\nTITLE: Me [19 F]...   
1  SUBREDDIT: r/Parenting\n\nTITLE: My 11 year ol...   
2  SUBREDDIT: r/relationships\n\nTITLE: The girl ...   
3  SUBREDDIT: r/tifu\n\nTITLE: TIFU by accidently...   
4  SUBREDDIT: r/relationships\n\nTITLE: I [32 M] ...   

                                          completion  
0   I really like this guy, but after having sex ...  
1   Sons good friend died and his funeral is toda...  
2   Girl I'm seeing didn't respond to my texts wh...  
3   Tried to stop an old lady falling, kicked her...  
4   Wife Cheats on me but I stuck around for kids...  
